# 0. 필요 라이브러리 다운로드

In [1]:
!pip install -q transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.4 MB/s eta 0:00:00


# 1. 코랩 <-> 드라이브 연결

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

# 2. 지정한 경로에서 Model, Tokenizer 가져오기 & 파이프라인 생성

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import textwrap

# 지정한 경로 가져옴
model_path = "/content/drive/MyDrive/DILAB/llama3-Korean-Bllossom-8B"

# 지정한 경로에서 토크나이저, Model들을 불러온다
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map = "auto", # GPU 자동 할당
    torch_dtype = "auto" # fp16 등 자동 감지
)

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",     # GPU 할당
)

# 3. 반환값 생성 함수 정의

In [ ]:

def generate_response(user_input, history, temperature=0.7, top_p=0.9, max_tokens=500):
  global pipe
  global his
  his = history
  print(history)
  messages = [{"role":"assistant", "content":"너는 뛰어난 AI 어시스턴트야. 너는 상대방과 대화를 하는 챗봇이야."}]

  for h in history:
    messages.append({"role":"user", "content": h[0]})
    messages.append({"role":"assistant", "content": h[1]})

  messages.append({"role":"user", "content": user_input})

  prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

  # 텍스트 생성
  output = pipe(
      prompt,
      max_new_tokens=max_tokens,     # 생성할 최대 토큰 수
      do_sample=True,        # 샘플링 방식 사용 (더 자연스럽게)
      temperature=temperature,       # 창의성 조절 (낮을수록 보수적)
      top_p=top_p,              # nucleus sampling
      eos_token_id=tokenizer.eos_token_id,  # 문장이 끝났음을 뜻하는 eos_token_id를 명시하여, 모델이 끝을 알게 하기
      return_full_text=False
  )
  # 출력 결과 확인
  output_text = output[0]["generated_text"]

  #wrapped_text = textwrap.fill(output_text, width=70)
  return output_text


# 4. Gradio 웹 UI 인터페이스 구축

In [ ]:
import gradio as gr

his = [] # 대화 기록 저장용 리스트

iface = gr.ChatInterface(
    fn=generate_response,
    title="Llama3 ChatBot",
    description = "Let's talking with me"
)

iface.launch(share = True)